In [ ]:
# To use louvain algorithm
!pip3 install python-louvain
# To use progressbar
!pip3 install progressbar2

In [1]:
import pandas as pd
import numpy as np
import math as m
import time 
from pycowview.data import csv_read_FA
from pycowview.manipulate import unique_cows
from pycowview.metrics import interaction_time
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
import os
import community
from collections import defaultdict,Counter
import progressbar
import random
import itertools 
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics import f1_score

In [2]:
# Weighted version!
# This function will get the path of each csv file
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            if f.endswith('.csv'):
                fullname = os.path.join(root, f)
                yield fullname

# Input is the folders where the time matrix and cowlist are saved
# Output is a list which consists of 14 dictionaries
# The structure of dictionary:Cowlist,TimeMatrix,AajacencyMatrix_binary,Unweighted_Graph
def time_matrix_to_graph(tm_folder,cl_folder):
    dict_list = []
    i = 0
    tmlist = list(findAllFile(tm_folder))
    tmlist.sort()
    cllist = list(findAllFile(cl_folder))
    cllist.sort()
    for tm,cl in zip(tmlist,cllist):
        # print(tm,cl)
        # Get the path of csv
        # get cowlist
        cowlist = np.loadtxt(cl,delimiter=",").astype(int)
        # load original time matrix from csv and process it to be an adjacency Matrix
        OM = np.asmatrix(np.loadtxt(tm,delimiter=","))
        
        # Get unweighted adjacency matrix(binary)
        # init adjacency matrix
        #AM = np.zeros((OM.shape))
        # set the threshold to be 30 minutes(1800 seconds)
        #epsilon = 1800
        # just consider if there is an edge between two cows, the edge is unweighted
        #AM[OM >= epsilon] = 1
        #AM[OM < epsilon] = 0 
        #np.fill_diagonal(AM,0)
        # Get graph from AM, no-direct and no-weight graph
        #G_AM_temp = nx.from_numpy_matrix(AM,parallel_edges=False,create_using = nx.Graph())
        
        # Get weighted adjacency matrix
        # load original matrix from csv and process it to be an adjacency Matrix
        AM = np.zeros((OM.shape))
        # set the threshold to be 30 minutes(1800 seconds)
        epsilon = 1800
        # consider if there is an edge between two cows, the edge is weighted
        maxnr=np.amax(OM)
        AM=np.where(OM<=epsilon,0,(OM-epsilon)/(maxnr-epsilon))
        np.fill_diagonal(AM,0)
        # Get graph from AM, no-direct and weighted graph
        G_AM_temp = nx.from_numpy_matrix(AM,parallel_edges=False,create_using = nx.Graph())
        
        # Make sure the order of cowlist is the same as the row name!
        print('Shape of matrix:',AM.shape)
        print('number of nodes in graph',len(G_AM_temp),'length of cowlist',len(cowlist))
        mapping = dict(zip(G_AM_temp, cowlist))
        #print(mapping)
        # Rename the nodes
        G_AM = nx.relabel_nodes(G_AM_temp, mapping)
        
        # Get the dict of the collection(CL,TM,AM_weighted,Graph)
        data_dict = dict(CL=cowlist,TM=OM,AM_weighted=AM,Graph=G_AM)
        print('Document No.',i)
        print('TM path:',tm,'CL path:',cl)
        i = i + 1
        dict_list.append(data_dict)

    print('The length of the list: ',len(dict_list))
    return dict_list

In [3]:
def community_Louvain(i,G):
    #print('This is the result of day %d'%(i+1))
    
    # Nodes whose degree is zero
    nodes_removed = [node for node,degree in dict(G.degree()).items() if degree == 0]
    # If you want to remove them
    #G.remove_nodes_from(nodes_removed)
    #print(len(nodes_removed),'nodes whose degree is zero are removed')
    
    # Louvain algorithm
    partition = community.best_partition(G,weight = 'weight',randomize=False)
    num_communities = max(partition.values())
    
    # create a dict object:{community1:[nodelist],community2:[nodelist],......}
    # community1:[nodelist] is a tuple
    communities_Louvain = defaultdict(list) 
    for k, v in partition.items():
        communities_Louvain[v].append(k)    
    np.save('./community/Louvain_weighted/Day_%d_Louvain_weighted_communities.npy'%(i+1), communities_Louvain)
    
    #read the dict from file
    #communities_Louvain = np.load('./community/Louvain/Day_%d_Louvain_Unweighted_communities.npy', allow_pickle='TRUE')
    #print(communities_Louvain)

    num_communities = max(partition.values())
    len_Louvain = len(communities_Louvain)
    #print('max No. of community:',num_communities)
    #print('num of communities:',len_Louvain)
    #print('Modularity:',nx.algorithms.community.quality.modularity(G,communities_Louvain.values()))

    # Colormap for plotting
    color_Louvain = 0
    random.seed(7)
    total_colors = list(mpl.colors.get_named_colors_mapping())
    total_colors.remove('black')
    color_map_Louvain = random.sample(total_colors,len_Louvain)
    
    '''
    # Plot the figure
    plt.figure(figsize=(15, 15))  # image size
    pos = nx.fruchterman_reingold_layout(G, scale = 1) # position of nodes
    degree_dict = dict(G.degree())
    nx.draw_networkx(G, pos, node_size=5,width=0.05, alpha=1, with_labels=False)
    for community_Louvain in communities_Louvain.items():
        
        node_list = community_Louvain[1]
        edge_list = list(itertools.chain.from_iterable([list(G.edges(node)) for node in node_list]))
        label_list = {}
        for node in node_list:
            #set the node name as the key and the label as its value 
            label_list[node] = node 
        community_degree_dict = {key: value for key, value in degree_dict.items() if key in node_list}
        node_size_list = [d*20 for d in community_degree_dict.values()]
        
        nx.draw_networkx_nodes(G, pos , nodelist = node_list, node_size = node_size_list, node_color = color_map_Louvain[color_Louvain],alpha = 0.7)
        nx.draw_networkx_edges(G, pos , edgelist = edge_list, edge_color = color_map_Louvain[color_Louvain],alpha = 0.2)
        nx.draw_networkx_labels(G, pos, label_list, font_size = 8, font_color = color_map_Louvain[color_Louvain],alpha = 0.7)
        color_Louvain += 1
    plt.savefig('./community/Louvain_weighted/Day%d_weighted_Community.png'%(i+1))
    #plt.show()
    '''
    return nodes_removed, communities_Louvain, partition

In [4]:
def compute_NMI(p1,p2):
    list1 = []
    list2 = []
    dict1 = {}
    dict2 = {}
    key1set = set(sorted(p1))
    key2set = set(sorted(p2))
    keylist = list(key1set&key2set)
    keylist.sort()
    #print(len(key1set))
    #print(len(key2set))
    #print(len(keylist))
    i = 0
    for key in keylist:
        list1.append(p1.get(key,-1))
        list2.append(p2.get(key,-1))
        dict1[key] = p1.get(key,-1)
        dict2[key] = p2.get(key,-1)
        #i = i+1
        #print(i,key)
    #print(list1)
    #print(list2)
    return normalized_mutual_info_score(list1,list2), adjusted_mutual_info_score(list1,list2), f1_score(list1,list2, average='weighted')

In [5]:
def create_rand_adjacency(day):
    # create a random adjacency matrix of a specific day
    # compute the density
    graph = data_dict_list[day-1].get('Graph')
    density = nx.classes.function.density(graph)
    # create a basic matrix has the density
    AM = data_dict_list[day-1].get('AM_weighted')
    AM_rand = np.random.rand(AM.shape[0]*AM.shape[1]).reshape((AM.shape))
    AM_temp = np.zeros((AM_rand.shape))
    AM_temp[AM_rand <= density] = 1
    AM_temp[AM_rand > density] = 0

    # unweighted
    AM_up = np.triu(AM_temp)
    np.fill_diagonal(AM_up,0)
    AM_unweighted_rand = AM_up + AM_up.T

    # weighted
    AM_weighted_up = np.zeros((AM_up.shape))
    row,col = np.where(AM_up == 1)
    for i,j in zip(row,col):
        if AM_up[i][j] == 1:
            AM_weighted_up[i][j] = np.random.rand(1)
        else:
            pass
    AM_weighted_rand = AM_weighted_up + AM_weighted_up.T
    
    return AM_unweighted_rand, AM_weighted_rand 

In [6]:
# This part is used to process the time matrices
tm_folder = './time_matrix'
cl_folder = './cow_list'
data_dict_list = time_matrix_to_graph(tm_folder,cl_folder)

Shape of matrix: (213, 213)
number of nodes in graph 213 length of cowlist 213
Document No. 0
TM path: ./time_matrix\Time_FA_20201016T000000UTC.csv CL path: ./cow_list\Cow_list_20201016T000000UTC.csv
Shape of matrix: (212, 212)
number of nodes in graph 212 length of cowlist 212
Document No. 1
TM path: ./time_matrix\Time_FA_20201017T000000UTC.csv CL path: ./cow_list\Cow_list_20201017T000000UTC.csv
Shape of matrix: (219, 219)
number of nodes in graph 219 length of cowlist 219
Document No. 2
TM path: ./time_matrix\Time_FA_20201018T000000UTC.csv CL path: ./cow_list\Cow_list_20201018T000000UTC.csv
Shape of matrix: (208, 208)
number of nodes in graph 208 length of cowlist 208
Document No. 3
TM path: ./time_matrix\Time_FA_20201019T000000UTC.csv CL path: ./cow_list\Cow_list_20201019T000000UTC.csv
Shape of matrix: (209, 209)
number of nodes in graph 209 length of cowlist 209
Document No. 4
TM path: ./time_matrix\Time_FA_20201020T000000UTC.csv CL path: ./cow_list\Cow_list_20201020T000000UTC.csv


# Similarity of weighted adjcaceny matrices

In [12]:
def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

In [13]:
for i in range(0,len(data_dict_list)-1):
    graph1 = data_dict_list[i].get('Graph')
    graph2 = data_dict_list[i+1].get('Graph')
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)

72.69912696399258
78.69469651391837
4.49341784971925
1.8695783986913195
35.04457880842874
1.2454578637871314
5.973805070833857
29.788064339285434
55.516592244954076
66.44244241788113
150.89953835489598
46.6541261406251
1.2164612363136527


In [7]:
# Louvain algorithm
partition_14days = []
Louvain_community_14days = []
nodes_removed_14days = []
bar = progressbar.ProgressBar()
# from 0 to 13
for i in bar(range(0,len(data_dict_list))):
    nodes_removed_1day, Louvain_community_1day, partition_1day = community_Louvain(i,data_dict_list[i].get('Graph'))
    nodes_removed_14days.append(nodes_removed_1day)
    Louvain_community_14days.append(Louvain_community_1day)
    partition_14days.append(partition_1day)

100% (14 of 14) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [8]:
# To test our custom functions using karate_club data
G1 = nx.karate_club_graph()
G2 = nx.karate_club_graph()
n1,l1,p1=community_Louvain(1,G1)
n2,l2,p2=community_Louvain(1,G2)
compute_NMI(p1,p2)

(1.0, 1.0, 1.0)

In [9]:
#real day j vs real day j+1
bar = progressbar.ProgressBar()
print('Weighted graph(Louvain)')
for j in bar(range(1,len(data_dict_list))):
    print('Real Day %d and Real Day %d :NMI is'%(j,j+1),compute_NMI(partition_14days[j-1],partition_14days[j]))

100% (13 of 13) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Weighted graph(Louvain)
Real Day 1 and Real Day 2 :NMI is (0.16759813447692676, 0.04399274450844319, 0.1301717791111283)
Real Day 2 and Real Day 3 :NMI is (0.11373515938702934, -0.002553685759412102, 0.08459330854622636)
Real Day 3 and Real Day 4 :NMI is (0.1216408649805718, -0.0015233012720143025, 0.10502459128947593)
Real Day 4 and Real Day 5 :NMI is (0.15503870578877993, 0.00017495018003366806, 0.07274456481222105)
Real Day 5 and Real Day 6 :NMI is (0.1629188089829455, 0.009438257593828516, 0.05437075269835287)
Real Day 6 and Real Day 7 :NMI is (0.17919773324331983, 0.028885450513215387, 0.06579838205329945)
Real Day 7 and Real Day 8 :NMI is (0.1735237194029516, 0.0347117623690085, 0.06617987302644973)
Real Day 8 and Real Day 9 :NMI is (0.15136977461103673, 0.03719017912849117, 0.10042835321291432)
Real Day 9 and Real Day 10 :NMI is (0.1340610112240058, 0.006813156249437027, 0.1040320393996427)
Real Day 10 and Real Day 11 :NMI is (0.1855130976519167, 0.0613918260605716, 0.0775782229

In [10]:
import numpy as np
#random day j vs random day j+1
bar = progressbar.ProgressBar()
#print('Weighted graph(Louvain)')
NMIAVG = np.zeros((1000,13))
for k in bar(range(1,1001)):
    
    for j in range(1,len(data_dict_list)):
        # random graph of one day(have the same density) j
        uwr1,wr1 = create_rand_adjacency(j)
        # Get graph from AM, no-direct and weighted graph
        G_AM_temp1 = nx.from_numpy_matrix(wr1,parallel_edges=False,create_using = nx.Graph())
        cowlist1 = data_dict_list[j-1].get('CL')
        # Make sure the order of cowlist is the same as the row name!
        #print('Shape of matrix:',wr.shape)
        #print('number of nodes in graph',len(G_AM_temp),'length of cowlist',len(cowlist))
        mapping1 = dict(zip(G_AM_temp1, cowlist1))
        #print(mapping)
        # Rename the nodes
        G_AM1 = nx.relabel_nodes(G_AM_temp1, mapping1)
        nodes_removed_1day_r1, Louvain_community_1day_r1, partition_1day_r1 = community_Louvain(j-1,G_AM1)

        # random graph of one day(have the same density) j
        uwr2,wr2 = create_rand_adjacency(j+1)
        # Get graph from AM, no-direct and weighted graph
        G_AM_temp2 = nx.from_numpy_matrix(wr2,parallel_edges=False,create_using = nx.Graph())
        cowlist2 = data_dict_list[j].get('CL')
        # Make sure the order of cowlist is the same as the row name!
        #print('Shape of matrix:',wr.shape)
        #print('number of nodes in graph',len(G_AM_temp),'length of cowlist',len(cowlist))
        mapping2 = dict(zip(G_AM_temp2, cowlist2))
        #print(mapping)
        # Rename the nodes
        G_AM2 = nx.relabel_nodes(G_AM_temp2, mapping2)
        nodes_removed_1day_r2, Louvain_community_1day_r2, partition_1day_r2 = community_Louvain(j,G_AM2)    
        n1,n2,n3 = compute_NMI(partition_1day_r1,partition_1day_r2)
        NMIAVG[k-1][j-1] = n1
        #print('Random Day %d and Random Day %d :NMI is'%(j,j+1),NMIAVG[k-1][j-1])
        
    
print('Finish！')
    #print('----------')
    #print('Real Day %d and Real Day %d :NMI is'%(j,j+1),compute_NMI(partition_14days[j-1],partition_14days[j]))
    #print('Random Day %d and Random Day %d :NMI is'%(j,j+1),compute_NMI(partition_1day_r1,partition_1day_r2))

100% (1000 of 1000) |####################| Elapsed Time: 0:25:53 Time:  0:25:53


Finish！


In [11]:
print('Two consecutive days & Mean & Variance & Standard deviation\\\\')
for j in range(1,len(data_dict_list)):
    arr = NMIAVG[:][j-1] 
    # 求均值
    arr_mean = np.mean(arr)
    # 求方差
    arr_var = np.var(arr)
    # 求总体标准差
    arr_std = np.std(arr)
    print('Day %d and %d:'%(j,j+1),'&',arr_mean,'&',arr_var,'&',arr_std,'\\\\')

Two consecutive days & Mean & Variance & Standard deviation\\
Day 1 and 2: & 0.10262179823394588 & 0.0002910790229389459 & 0.01706103815536868 \\
Day 2 and 3: & 0.09787934905484656 & 0.00028975116305991256 & 0.017022078693858533 \\
Day 3 and 4: & 0.10767940766149867 & 0.00030795021095357815 & 0.017548510220345717 \\
Day 4 and 5: & 0.10641924962003899 & 0.00022439533238946833 & 0.014979830853166145 \\
Day 5 and 6: & 0.10544164150520333 & 0.00046778747468641246 & 0.02162839510195827 \\
Day 6 and 7: & 0.10864918915253 & 0.00035731038343726036 & 0.01890265545994161 \\
Day 7 and 8: & 0.10385102787881653 & 0.00039576101173239425 & 0.01989374302971651 \\
Day 8 and 9: & 0.12068617048025368 & 0.0003887798949074108 & 0.019717502248190837 \\
Day 9 and 10: & 0.11090307669255885 & 0.0005351663318646759 & 0.023133662309817612 \\
Day 10 and 11: & 0.11314365487249993 & 0.00021021583166667423 & 0.01449882173373665 \\
Day 11 and 12: & 0.12260520686023647 & 0.0002952671871276667 & 0.01718334039491934 \\


In [ ]:
# Random list NMI
bar = progressbar.ProgressBar()
print('Random list NMI')
for j in bar(range(1,len(data_dict_list))):
    np.random.seed()
    list1 = np.random.randint(low = 0, high = 11, size = 210)
    np.random.seed()
    list2 = np.random.randint(low = 0, high = 11, size = 210)
    NMI = normalized_mutual_info_score(list1,list2)
    print('NMI of random test %d'%(j),NMI)

In [ ]:
# random graph of day 
uwr,wr = create_rand_adjacency(6)
# Get graph from AM, no-direct and weighted graph
G_AM_temp = nx.from_numpy_matrix(wr,parallel_edges=False,create_using = nx.Graph())
cowlist = data_dict_list[5].get('CL')
# Make sure the order of cowlist is the same as the row name!
print('Shape of matrix:',wr.shape)
print('number of nodes in graph',len(G_AM_temp),'length of cowlist',len(cowlist))
mapping = dict(zip(G_AM_temp, cowlist))
#print(mapping)
# Rename the nodes
G_AM = nx.relabel_nodes(G_AM_temp, mapping)
nodes_removed_1day_r, Louvain_community_1day_r, partition_1day_r = community_Louvain(5,G_AM)

In [ ]:
# The NMI of real day1 and real day2
list1 = []
list2 = []
key1set = set(sorted(partition_14days[0]))
key2set = set(sorted(partition_14days[1]))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[0].get(key,-1))
    list2.append(partition_14days[1].get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# The NMI of real day1 and random day1
list1 = []
list2 = []
key1set = set(sorted(partition_14days[0]))
key2set = set(sorted(partition_1day_r))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[0].get(key,-1))
    list2.append(partition_1day_r.get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# The NMI of real day2 and random day1
list1 = []
list2 = []
key1set = set(sorted(partition_14days[1]))
key2set = set(sorted(partition_1day_r))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[1].get(key,-1))
    list2.append(partition_1day_r.get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# random graph of day 13
uwr13,wr13 = create_rand_adjacency(13)
# Get graph from AM, no-direct and weighted graph
G_AM_temp = nx.from_numpy_matrix(wr13,parallel_edges=False,create_using = nx.Graph())
cowlist = data_dict_list[12].get('CL')
# Make sure the order of cowlist is the same as the row name!
print('Shape of matrix:',wr13.shape)
print('number of nodes in graph',len(G_AM_temp),'length of cowlist',len(cowlist))
mapping = dict(zip(G_AM_temp, cowlist))
#print(mapping)
# Rename the nodes
G_AM = nx.relabel_nodes(G_AM_temp, mapping)
nodes_removed_1day_r, Louvain_community_1day_r, partition_1day_r = community_Louvain(0,G_AM)

In [ ]:
# The NMI of real day13 and random day13
list1 = []
list2 = []
key1set = set(sorted(partition_14days[12]))
key2set = set(sorted(partition_1day_r))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[12].get(key,-1))
    list2.append(partition_1day_r.get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# The NMI of real day14 and random day13
list1 = []
list2 = []
key1set = set(sorted(partition_14days[13]))
key2set = set(sorted(partition_1day_r))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[13].get(key,-1))
    list2.append(partition_1day_r.get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# The NMI of real day14 and real day13
list1 = []
list2 = []
key1set = set(sorted(partition_14days[13]))
key2set = set(sorted(partition_14days[12]))
keylist = list(key1set&key2set)
print(len(key1set))
print(len(key2set))
print(len(keylist))
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[13].get(key,-1))
    list2.append(partition_14days[12].get(key,-1))
    #i = i+1
    #print(i,key)
#print(list1)
#print(list2)
normalized_mutual_info_score(list1,list2)

In [ ]:
# check the first day community
for c,n in Louvain_community_14days[0].items():
    n.sort()
    print('community',c)
    print('number of nodes in community',len(n))
    print(n)

In [ ]:
for c,n in Louvain_community_14days[1].items():
    n.sort()
    print('community',c)
    print('number of nodes in community',len(n))
    print(n)

In [ ]:
'''
def compute_NMI(day1,day2):
    list1 = list(partition_14days[day1-1].values())
    list2 = list(partition_14days[day2-1].values())
    if len(list1)<len(list2):
        rand = np.random.randint(low = 0, high = max(list1)+1, size = len(list2)-len(list1))
        list1.extend(rand)
    elif len(list2)<len(list1):
        rand = np.random.randint(low = 0, high = max(list2)+1, size = len(list1)-len(list2))
        list2.extend(rand)
    else:
        pass
    print(len(list1),len(list2))
    return normalized_mutual_info_score(list1,list2)
'''

In [ ]:
'''
Draft
rand = np.random.randint(low = 0, high = max(partition_14days[0].values())+1, size = 1)
print(partition_14days[0].get(999,rand[0]))
list1=[]
list2=[]
key1set = set(sorted(partition_14days[0]))
key2set = set(sorted(partition_14days[1]))
keylist = list(key1set&key2set)
keylist.sort()
i = 0
for key in keylist:
    list1.append(partition_14days[0].get(key,-1))
    list2.append(partition_14days[1].get(key,-1))
    #i = i+1
    #print(i,key)
normalized_mutual_info_score(list1,list2)
'''